In [4]:
import networkx as nx
import numpy as np  

In [5]:
def spell(word, first, second):  # word is either 'All' or 'Some' or 'At least'
    pvar1 = nodes[first]
    pvar2 = nodes[second]
    if pvar1[1] == pos and pvar2[1] == neg and word == 'All':
        return ['No', simple(pvar1), 'are', simple(negation(pvar2))]
    elif word == 'At least':
        return ['At least as many', simple(pvar1), 'as', simple(pvar2)]
    elif word == 'More':
        return ['There are more', simple(pvar1), 'than', simple(pvar2)]
    elif word == 'All' or word == 'Some':
        return [word, simple(pvar1), 'are', simple(pvar2)]
    #if pvar1[1] == pos and pvar2[1] == pos and (word == 'All' or  word == 'Some'):
    #    return [word, pvar1[0], 'are', pvar2[0]]

    #if pvar1[1] == pos and pvar2[1] == neg and word == 'Some':
    #    return ['Some', pvar1[0], 'are', ('non',pvar2[0])]
    #if pvar1[1] == neg and pvar2[1] == pos:
    #    return [word, ('non',pvar1[0]), 'are', pvar2[0]]
    #if pvar1[1] == neg and pvar2[1] == neg:
    #    return [word, ('non',pvar1[0]), 'are', ('non',pvar2[0])]
    else:
        print("error in spell")

In [6]:
pos = "+"
neg = "-"

def negation(argument):
    if argument[1] == pos:
        return ([argument[0], neg])
    if argument[1] == neg:
        return ([argument[0], pos])

def sentence_negation(phi):
    if phi[0] == 'All':
        return ['Some', phi[1], negation(phi[2])]
    if phi[0] == 'Some':
        return ['All', phi[1], negation(phi[2])]
    if phi[0] == 'At least':
        return ['More', phi[2], phi[1]]
    if phi[0] == 'More':
        return ['At least', phi[2], phi[1]]

def last(v):
    return (v[len(v) - 1])

def simple(pvar):
    if pvar[1] == pos:
        return pvar[0]
    else:
        return 'non-' + pvar[0]

In [7]:
def pre_process(assumption):
    assump = assumption.split(" ")
    if assump[0] == 'There' and assump[1] == 'are':
        assump = assump[2:]
    elif assump[0] == 'At':
        assump = [assump[0] + " " + assump[1]] + assump[2:]
    elif assump[0] == 'at':
        assump = ['At' + " " + assump[1]] + assump[2:]
    elif assump[1] == 'as' and assump[2] == 'many':
        assump = [assump[0]] + assump[3:]
    elif assump[0] == 'more':
        assump = ['More'] + assump[1:]
    if len(assump[1]) == 1:
        predicate1 = assump[1]
        argument1 = (predicate1, pos)
    else:
        argument1 = (last(assump[1]), neg)
    if len(assump[3]) == 1:
        predicate2 = assump[3]
        argument2 = (predicate2, pos)
    else:
        argument2 = (last(assump[3]), neg)
    if assump[0] == 'No':
        assump[0] = 'All'
        dd = negation(dd)
    return [assump[0], argument1, argument2]

In [13]:
from itertools import product

def cartesian_product(arr1, arr2): 
    return list(product(arr1, arr2))  

def make_variables(Gamma):
    polarities = [pos, neg]
    vset = set([])
    for item in Gamma:
        vset = vset.union(set([item[1][0], item[2][0]]))
    variables = list(vset)
    variables.sort()
    polarized_variables = list(cartesian_product(variables, polarities))
    return variables, polarized_variables

In [51]:
def follows(asumptions, conclusion):
    Gamma = []
    for assumption in asumptions:
        Gamma += [pre_process(assumption)]
    phi = pre_process(conclusion)

    variables, pvars = make_variables(Gamma)
    dg = nx.DiGraph()
    dg.add_nodes_from(pvars)
    dgc = nx.DiGraph()
    dgc.add_nodes_from(pvars)

    dim = len(dg.nodes)

    mgs, proof_lens, justifications = read(Gamma, dim, list(dg.nodes))
    make_matrices(Gamma, mgs, proof_lens, justifications, dim)

    #sdagger(Gamma, phi)

In [52]:
def read(Gamma, dim, nodes):
    mg = np.zeros((dim, dim))
    mgc = np.zeros((dim, dim))
    mgsome = np.zeros((dim, dim))
    mgmore = np.zeros((dim, dim))
    justification = np.zeros((dim, dim))
    justificationc = np.zeros((dim, dim))
    justificationsome = np.zeros((dim, dim))
    justificationmore = np.zeros((dim, dim))
    proof_length = np.zeros((dim, dim))
    proof_lengthc = np.zeros((dim, dim))
    proof_lengthsome = np.zeros((dim, dim))
    proof_lengthmore = np.zeros((dim, dim))
    Barbara_cut_premise = np.zeros((dim, dim))
    card_transitivity_cut_premise = np.zeros((dim, dim))
    Card_Some_missing = np.zeros((dim, dim))
    More_Left_cut_premise = np.zeros((dim, dim))
    More_Right_cut_premise = np.zeros((dim, dim))
    Some_rule_missing_premise = np.zeros((1, dim))
    Darii_missing = np.zeros((dim, dim))
    Int_rule_missing_premise = np.zeros((1, dim))

    for item in Gamma:
        if item[0] == 'All':
            i = nodes.index(item[1])
            j = nodes.index(item[2])
            mg[i, j] = 1
            justification[i, j] = 2
            proof_length[i, j] = 1
        elif item[0] == 'At least':
            ## At least as many i as j  ===>   mgc[j,i] = 1
            ## That is, bigger sets are the targets
            i = nodes.index(item[1])
            j = nodes.index(item[2])
            mgc[j, i] = 1
            justificationc[j, i] = 2
            proof_lengthc[j, i] = 1
        elif item[0] == 'Some':
            i = nodes.index(item[1])
            j = nodes.index(item[2])
            mgsome[i, j] = 1
            justificationsome[i, j] = 2
            proof_lengthsome[i, j] = 1
        elif item[0] == 'More':
            i = nodes.index(item[1])
            j = nodes.index(item[2])
            mgmore[j, i] = 1
            justificationmore[j, i] = 2
            proof_lengthmore[j, i] = 1
        else:
            print("ERROR READING GAMMA")

    mgs = {
        "mg": mg, 
        "mgc": mgc, 
        "mgsome": mgsome, 
        "mgmore": mgmore
    }
    justifications = {
        "justification": justification,
        "justificationc": justificationc,
        "justificationsome": justificationsome,
        "justificationmore": justificationmore
    }
    proof_lens = {
        "proof_length": proof_length, 
        "proof_lengthc": proof_lengthc, 
        "proof_lengthsome": proof_lengthsome, 
        "proof_lengthmore": proof_lengthmore}

    return mgs, proof_lens, justifications

In [ ]:
def exists(arr, criteria):
   return map(criteria, arr)

def apply_using_numbers(mgs, graphs, matrices, justifications, dim):
    for i in range(dim):
        for j in range(dim):
            if justifications['justification'][i,j] == 0:
                k_found = exists(range(n), lambda k:
                                 (mgc[i,k] == 1 and mgc[k,j] == 1))
                if k_found[0] == True:
                    k = k_found[1]
                    justifications['justificationc'][i,j] = 12
                    mgs['mgc'][i,j] = 1
                    proof_lenths['proof_lengthc'][i,j] = 
                        1 + proof_lengthc[i,k] + proof_lengthc[k, j]
                    card_transitivity_cut_premise[i,j] = k
    

In [53]:
def make_matrices(Gamma, mgs, proof_lens, justifications, dim):
    small_or_half = []
    large_or_hald = []
    large_or_half_original = []
    small = []
    large = []
    half = []
    for i in range(dim):
        mgs['mg'][i,i] = 1
        justifications['justification'][i,i] = 1
        proof_lens['proof_length'][i,i] = 1

    old_max = -1
    new_max = 0

    while old_max < new_max:
        old_max = new_max 
        Apply_suing_numbers()
        m1 = proof_lens['proof_length'].height()
        m2 = proof_lens['proof_length'].height()
        m3 = proof_lens['proof_lengthsome'].height()
        m4 = proof_lens['proof_lengthmore'].height()
        new_max = max([m1, m2, m3, m4])

In [54]:
follows(["All A are B"], "All B are A")

NameError: name 'Apply_suing_numbers' is not defined